In [15]:
import pandas as pd

In [16]:
# Replace the path below with the actual path to your CSV file if it's not in the same directory as your Jupyter Notebook
file_path = 'commodities_data.csv'

# Read the CSV file
df = pd.read_csv(file_path)

In [17]:
# Convert the datetime column to a datetime object
#df['datetime'] = pd.to_datetime(df['datetime'], format='%Y-%m-%d %H:%M')

# Convert the 'datetime' column to datetime objects
df['datetime'] = pd.to_datetime(df['datetime'])

In [18]:
# Display the first few rows of the DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13254799 entries, 0 to 13254798
Data columns (total 8 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          int64         
 1   quantity    int64         
 2   unit_price  int64         
 3   time_left   object        
 4   item.id     int64         
 5   item.name   object        
 6   item.class  object        
 7   datetime    datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 809.0+ MB


In [19]:
df.head()

,id,quantity,unit_price,time_left,item.id,item.name,item.class,datetime
0,1883899371,2,237500,SHORT,38376,Heavy Borean Armor Kit,Consumables,2023-12-24 13:41:00
1,1883899383,1,14503400,SHORT,199973,Enchant Weapon - Frozen Writ,Unknown,2023-12-24 13:41:00
2,1883899456,5,47500,SHORT,38375,Borean Armor Kit,Consumables,2023-12-24 13:41:00
3,1883899469,54,46900,SHORT,154165,Calcified Bone,Trade Goods,2023-12-24 13:41:00
4,1883899485,23,4900,SHORT,38787,Enchant Shield - Minor Stamina,Consumables,2023-12-24 13:41:00


In [20]:
# Group by 'item.class' and count unique 'item.id'
class_counts = df.groupby('item.class')['item.id'].nunique()

print("Number of unique items for each item class:")
print(class_counts)

Number of unique items for each item class:
item.class
Armor              11
Consumables      3120
Gems              990
Glyphs            111
Miscellaneous    1799
Recipes             3
Trade Goods      2091
Unknown           340
Name: item.id, dtype: int64
